In [1]:
from __future__ import print_function, division
import numpy as np
from scikits.odes import dae
import pandas as pd
import time
import cantera as ct
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.family'] = 'Times New Roman'
colors = brewer2mpl.get_map('Set2','qualitative',8).mpl_colors

# Method 1

In [3]:
mech = '/Users/yuanjie/Dropbox/Cantera_intern/code/gas_surf_phase/gas_surf_bulk/mec.cti'
#import the models for gas and bulk
gas, bulk_Si, bulk_N = ct.import_phases(mech,['gas','SiBulk','NBulk',])

#import the model for gas-Si-N interface
gas_Si_N_interface = ct.Interface(mech, 'SI3N4',[gas,bulk_Si,bulk_N])

T = 1713 #K
p = 2 * ct.one_atm / 760.0 #Pa ~2Torr
gas.TPX = T,p,"NH3:6, SiF4:1"
gas_Si_N_interface.advance_coverages(10.0)
gas_Si_N_interface.coverages #which is different from the results in the paper
"""
Paper's results
HN_SIF(S)   = 6.242E-02
H2NFSINH(S) = 2.411E-04
F3SI_NH2(S) = 3.136E-04  
HN(FSINH)2(S)= 4.821E-04
F2SINH(S) = 2.081E-02
HN_NH2(S) = 9.157E-01
"""

array([  6.26078544e-02,   9.16094285e-01,   1.31554573e-04,
         2.08637653e-02,   1.00847012e-04,   2.01694023e-04])

# Method 2

In [9]:
mech = '/Users/yuanjie/Dropbox/Cantera_intern/code/gas_surf_phase/gas_surf_bulk/mec.cti'
#import the models for gas and bulk
gas, bulk_Si, bulk_N = ct.import_phases(mech,['gas','SiBulk','NBulk',])

#import the model for gas-Si-N interface
gas_Si_N_interface = ct.Interface(mech, 'SI3N4',[gas,bulk_Si,bulk_N])

T = 1713 #K
p = 2 * ct.one_atm / 760.0 #Pa ~2Torr
gas.TPX = T,p,"NH3:6, SiF4:1"
iC_tot = []
#get the index of the surface species
for surf_spec in ['HN_SIF(S)','F3SI_NH2(S)','F2SINH(S)','H2NFSINH(S)','HN(FSINH)2(S)','HN_NH2(S)']:
    iC = gas_Si_N_interface.kinetics_species_index(surf_spec)
    iC_tot.append(iC)

#calculate the net production rate of each surface species
s_dot = []
for ic in iC_tot:
    s_dot_k = gas_Si_N_interface.net_production_rates[ic]
    s_dot.append(s_dot_k)

#calculate the initial values of Z_k by using the factitious ransient equation
dt = 0.01
gamma = 0.417E-7 #site density kmol/m2
n_step = 100 #number of steps for time
Z = np.zeros((n_step,6)) #initialize Z
Z[0,:] = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6] #initial guess of Z_k_0
sigma = [2, 2, 2, 2, 4, 2] # site occupancy number
for n in range(n_step-1):
    for k in range(6):
        Z[n+1,k] = Z[n,k] + dt*s_dot[k]*sigma[k]/gamma
    if np.sum(Z[n,:])-1< 10**-8:
        break

In [10]:
Z #seems not correct

array([[ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [-1.93372381,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         2.26705715],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.